In [25]:
import pandas as pd
import numpy as np

In [26]:
GSS = pd.read_csv("GSS.csv")
pd.set_option('display.max_columns', None)
print(GSS.shape)
GSS.head(2)

# Selecting relevant variables -> for future codes, make selection of variables more versatile 

relevant_columns = ['year', 'id_', 'ballot', # Identifiers
    'happy', 'satjob', 'life', 'health', 'satfin',  # Satisfaction-related columns
    'age', 'sex', 'race', 'marital', 'educ', 'region',  # Demographics
    'wrktype', 'wrksched', 'hrs2', 'weekswrk', 'indus10', 'occ10', 'occ10_code', 'MPROFF', 'MPROFF_Label', 'conrinc', 'wrkslf',  # Work info
    'jobpromo', 'learnnew', 'workfast', 'opdevel', 'fairearn']  # Job quality

GSS_filtered= GSS[relevant_columns]
GSS_filtered.shape

(17749, 60)


(17749, 30)

Data Cleaning

In [27]:
non_response_values = [".i:  Inapplicable", ".n:  No answer", ".d:  Do not Know/Cannot Choose", ".s:  Skipped on Web"]
filtered_df = GSS_filtered.replace(non_response_values, np.nan)
filtered_df = filtered_df[filtered_df['wrktype'].notna()]  # This is our primary dependent variable, so we need responses to this variable
filtered_df.shape

(9255, 30)

In [28]:
missing_percent = filtered_df.isna().mean()
pd.DataFrame(missing_percent)

,0
year,0.000000
id_,0.000000
ballot,0.000000
happy,0.097461
satjob,0.096056
life,0.406483
health,0.266559
satfin,0.097893
age,0.013830
sex,0.000540


In [37]:
# Drop variables with high missingness (>20%)
filtered_df_cleaned = filtered_df.drop(['life', 'health', 'hrs2', 'jobpromo'], axis=1)
print(filtered_df_cleaned.shape)

# To ensure analyses are based on complete survey responses, remove all remaining rows with missing values
filtered_df_cleaned = filtered_df_cleaned.dropna()
print(filtered_df_cleaned.shape)

(9255, 26)
(7742, 26)


In [32]:
missing_percent_2= drop_high.isna().mean()
missing_percent_2

year            0.000000
id_             0.000000
ballot          0.000000
happy           0.097461
satjob          0.096056
satfin          0.097893
age             0.013830
sex             0.000540
race            0.002809
marital         0.000864
educ            0.002053
region          0.000000
wrktype         0.000000
wrksched        0.002917
weekswrk        0.010481
indus10         0.009724
occ10           0.009508
occ10_code      0.009508
MPROFF          0.000000
MPROFF_Label    0.000000
conrinc         0.000000
wrkslf          0.000756
learnnew        0.004430
workfast        0.005943
opdevel         0.010589
fairearn        0.027337
dtype: float64